In [1]:
!gdown https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL
!unzip /content/Project_data.zip

Streaming output truncated to the last 5000 lines.
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00050.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00052.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00054.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00056.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00058.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00060.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00062.png  
   creating: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/
 extracting: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/WIN_20180926_17_35_12_Pro_00001.png  
 extracting: Project_data/t

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd

import random as rn
import numpy.random as nrn
import tensorflow as tf
import os

import keras
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers import Conv3D, MaxPooling3D, Dropout
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

import PIL
from PIL import Image
import cv2

import matplotlib.pyplot as plt

In [2]:
rn.seed(37)
nrn.seed(37)
tf.random.set_seed(37)

In [3]:
data_folder = "/content/Project_data"

train_df = pd.read_csv(f"{data_folder}/train.csv", sep=";", header=None)
train_df.columns = ['folder', 'type', 'class']
val_df = pd.read_csv(f"{data_folder}/val.csv", sep=";", header=None)
val_df.columns = ['folder', 'type', 'class']

In [4]:
def get_folder_list(df_csv):
  fptr = open(df_csv)
  folder_paths = fptr.readlines()
  fptr.close()
  return folder_paths

def get_batch_details(folder_list, batch_size):
  num_batches = len(folder_list) // batch_size
  folders_remain = len(folder_list) - (num_batches * batch_size)
  return num_batches, folders_remain

def create_batch_folders(batch_size, img_ids, image_shape):
         ## batch_images                                                                               ## batch_labels
  return np.zeros(shape = (batch_size, len(img_ids), image_shape[1], image_shape[2], image_shape[3])), np.zeros(shape = (batch_size, 5))

def process_images(folder, folder_path, img_ids, image_shape, batch_images):
  imgs = os.listdir(f'{folder_path.split(";")[0]}')
  for idx, ind in enumerate(img_ids):
    ## read image
    image = cv2.imread(f'{folder_path.split(";")[0]}/{imgs[ind]}').astype(np.float32)
    ## resize image
    image = cv2.resize(image, (image_shape[1], image_shape[2]), interpolation = cv2.INTER_LINEAR)
    ## Seperating them into their respective R, G, B channels and normalizing the values
    ## Load labels for each image
    batch_images[folder, idx, :, :, 0] = image[:,:,0]/255.0   ## R
    batch_images[folder, idx, :, :, 1] = image[:,:,1]/255.0   ## G
    batch_images[folder, idx, :, :, 2] = image[:,:,2]/255.0   ## B

In [5]:
## path => folder path as string
## folder_list => list of folders in train/test folder as list
## batch_size => chosen batch size as int
## image_shape => as tuple (image_height, image_width)
def run_generator(path, folder_list, batch_size, image_shape):
  num_batches, folders_remain = get_batch_details(folder_list, batch_size)
  ## set how many images to view from total images of the video
  img_ids = list(range(0,30,2))
  while True:
    folder_list = np.random.permutation(folder_list)
    for batch in range(num_batches):
      batch_images, batch_labels = create_batch_folders(batch_size, img_ids, image_shape)
      ## get folders for that batch
      for folder in range(batch_size):
        process_images(folder, f'{path}/{folder_list[folder + (batch * batch_size)]}', img_ids, image_shape, batch_images)
      ## get images and labels for each folder in the batch
      #####################################
          ## Load labels for each image
        batch_labels[folder, int(folder_list[folder + (batch * batch_size)].strip().split(';')[2])] = 1
      yield batch_images, batch_labels

    ## Process the remaining folders
    batch_images, batch_labels = create_batch_folders(folders_remain, img_ids, image_shape)
    folder_rem = folder_list[-folders_remain:]
    for folder in range(folders_remain):
      process_images(folder, f'{path}/{folder_rem[folder]}', img_ids, image_shape, batch_images)
      ####################################
      batch_labels[folder, int(folder_rem[folder].strip().split(';')[2])] = 1
    yield batch_images, batch_labels

In [6]:
curr_dt_time = datetime.now()

model_name = 'model_cnn3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_lr=0.001)
callbacks_list = [checkpoint, LR]

In [7]:
# train_folders = []
# val_folders = []
# for i in range(5):
#   train_folders.extend(train_df.loc[train_df['class'] == i, ['folder', 'class']][:20])
#   val_folders.extend(train_df.loc[train_df['class'] == i, ['folder', 'class']][-5:])

In [8]:
fptr = open('/content/Project_data/train.csv')
train_paths = fptr.readlines()
fptr.close()
train_folder_list_ = np.random.permutation(train_paths)


fptr = open('/content/Project_data/val.csv')
val_paths = fptr.readlines()
fptr.close()
val_folder_list_ = np.random.permutation(val_paths)


In [9]:
train_folder_list = train_folder_list_[:150]
val_folder_list = val_folder_list_[-50:]

In [10]:
# train_folder_list = np.random.permutation(train_folder_list)
# val_folder_list = np.random.permutation(val_folder_list)
# len(train_folder_list), len(val_folder_list)

In [11]:
batch_size = 5
num_epochs = 20
image_shape = (15, 224, 224, 3)

In [12]:
if (len(train_folder_list)%batch_size) == 0:
    steps_per_epoch = int(len(train_folder_list)/batch_size)
else:
    steps_per_epoch = int(len(train_folder_list)//batch_size) + 1

if (len(val_folder_list)%batch_size) == 0:
    validation_steps = int(len(val_folder_list)/batch_size)
else:
    validation_steps = int(len(val_folder_list)//batch_size) + 1

In [13]:
train_generator = run_generator(f'{data_folder}/train', train_folder_list, batch_size, image_shape)
val_generator = run_generator(f'{data_folder}/val', val_folder_list, batch_size, image_shape)

In [ ]:
  # model.add(layers.Conv3D(32, (3, 3, 3), activation='relu')),
  # model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),
  # model.add(keras.layers.BatchNormalization()),

  # model.add(layers.Conv3D(64, (3, 3, 3), activation='relu')),
  # model.add(layers.Conv3D(64, (3, 3, 3), activation='relu')),
  # model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),
  # model.add(keras.layers.BatchNormalization()),

  model.add(layers.Conv3D(128, (3, 3, 3), activation='relu')),

  # model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),

  # model.add(layers.Conv3D(256, (3, 3, 3), activation='relu')),
  # model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),
  # model.add(layers.Conv3D(512, (3, 3, 3), activation='relu')),

In [34]:
def cnn3D_model(vid_shape):
  model = Sequential()

  model.add(layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=vid_shape)),
  model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),
  model.add(keras.layers.BatchNormalization()),

  model.add(layers.Conv3D(64, (3, 3, 3), activation='relu', input_shape=vid_shape)),
  model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),
  model.add(keras.layers.BatchNormalization()),

  model.add(layers.Conv3D(128, (3, 3, 3), activation='relu', input_shape=vid_shape)),
  model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),
  model.add(keras.layers.BatchNormalization()),

  model.add(layers.Conv3D(256, (3, 3, 3), activation='relu', input_shape=vid_shape)),
  model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),
  model.add(keras.layers.BatchNormalization()),

  model.add(layers.Conv3D(512, (3, 3, 3), activation='relu', input_shape=vid_shape)),

  ## Flatten and Dense layers
  model.add(layers.GlobalAveragePooling3D()),
  model.add(layers.Dense(1024, activation='relu')),
  model.add(layers.Dropout(0.4)),
  model.add(layers.Dense(1024, activation='relu')),
  model.add(layers.Dense(5, activation='softmax'))

  return model

In [35]:
model = cnn3D_model(image_shape)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d_47 (Conv3D)                   │ (None, 13, 222, 222, 32)    │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_24 (MaxPooling3D)      │ (None, 13, 111, 111, 32)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 13, 111, 111, 32)    │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_48 (Conv3D)                   │ (None, 11, 109, 109, 64)    │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_25 (MaxPooling3D)      │ (None, 11, 54, 54, 64)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 11, 54, 54, 64)      │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_49 (Conv3D)                   │ (None, 9, 52, 52, 128)      │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_26 (MaxPooling3D)      │ (None, 9, 26, 26, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 9, 26, 26, 128)      │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_50 (Conv3D)                   │ (None, 7, 24, 24, 256)      │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_27 (MaxPooling3D)      │ (None, 7, 12, 12, 256)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 7, 12, 12, 256)      │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_51 (Conv3D)                   │ (None, 5, 10, 10, 512)      │       3,539,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling3d_8           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling3D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 5)                   │           5,125 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,285,701 (23.98 MB)

 Trainable params: 6,284,741 (23.97 MB)

 Non-trainable params: 960 (3.75 KB)

In [ ]:
## Input + 32 + Dense_2(1024)
## Input + 32 + maxpool + 64 + Dense_2(1024)
## Input + 32 + 32 + maxpool + 64 + Dense_2(1024)
## Input + 32 + 32 + maxpool + 64 + 64 + Dense_2(1024)
## Input + 32 + 32 + maxpool + 64 + 64  + maxpool + 128 + Dense_2(1024)
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Epoch 1/20
 5/30 ━━━━━━━━━━━━━━━━━━━━ 12:07 29s/step - categorical_accuracy: 0.1847 - loss: 2.5562